In [44]:
# Run this first

import pandas as pd
import time
from IPython.display import clear_output

In [45]:
streaming_services = {'hulu':"data/hulu_titles.csv", \
                      'amazon_prime':"data/amazon_prime_titles.csv", \
                      'netflix':"data/netflix_titles.csv", \
                      'disney_plus':"data/disney_plus_titles.csv"}

df_list = list()

for ss in streaming_services.keys():
    ss_df = pd.read_csv(streaming_services[ss])
    ss_df['streaming_service'] = ss
    df_list.append(ss_df)

df = pd.concat(df_list)

df.to_csv("data/streaming_titles_prelim.csv",index=False)


In [47]:
import requests
print('start')
rapid_api_key = '5d5be84e09msh3c94417204a3bbbp149df0jsn7dfa2269ae85'
def imdbpuller(rapid_api_key,title,year):
    url = "https://mdblist.p.rapidapi.com/"

    querystring = {"s":title,"y":year}

    headers = {
        "X-RapidAPI-Key": rapid_api_key,
        "X-RapidAPI-Host": "mdblist.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return response


start


In [48]:
print(imdbpuller(rapid_api_key,"Breaking Bad",2008).json()["search"][0])

{'id': 'tt0903747', 'title': 'Breaking Bad', 'year': 2008, 'score': 92, 'score_average': 92, 'type': 'show', 'imdbid': 'tt0903747', 'tmdbid': 1396, 'traktid': 1388, 'tvdbid': 81189}


In [70]:
# ***DO NOT RUN***
# unless you want to restart computing streaming_titles.csv (will take a very long time)
# if you accidently run, just don't commit to main

df['score'] = None
df['imdbid'] = None
df.to_csv("data/streaming_titles.csv",index=False)


In [74]:
df2 = pd.read_csv("data/streaming_titles.csv",dtype={'score' : float, 'imdbid' : 'string'})

index = df2['score'].last_valid_index()
if index == None:
    index = 0

count = 0
for k in range(index,len(df2)):
    row = df2.iloc[k]
    title = row["title"]
    year = row["release_year"]
    responded = False
    respcount = 0

    while not responded:
        if respcount >= 10:
            clear_output()
            print("no response for {title}".format(title=title))
            break
        currjson = imdbpuller(rapid_api_key,title,year).json()
        responded = currjson['response']
        if not responded:
            time.sleep(0.25)
        respcount += 1
    if not responded:
        continue

    madeit = False
    for queries in currjson['search']:
        if queries['title'] == title:
            df2.at[k,"score"] = queries["score"]
            df2.at[k,"imdbid"] = queries["imdbid"]
            clear_output()
            print('success for {title}'.format(title=title))
            madeit = True
            break
    if not madeit:
        clear_output()
        print("failed for {title}".format(title=title))

    count += 1

    if count%20 == 0:
        clear_output()
        print("writing to csv... {index}".format(index=k))
        df2.to_csv("data/streaming_titles.csv",index=False)



failed for Down To Earth: The Astronaut Perspective


KeyboardInterrupt: 

In [67]:
print(df2)

      show_id   type                                           title  \
0          s1  Movie                  Ricky Velez: Here's Everything   
1          s2  Movie                                    Silent Night   
2          s3  Movie                                    The Marksman   
3          s4  Movie                                            Gaia   
4          s5  Movie                                        Settlers   
...       ...    ...                                             ...   
22993   s1446  Movie                        X-Men Origins: Wolverine   
22994   s1447  Movie  Night at the Museum: Battle of the Smithsonian   
22995   s1448  Movie                                 Eddie the Eagle   
22996   s1449  Movie                            Bend It Like Beckham   
22997   s1450  Movie           Captain Sparky vs. The Flying Saucers   

              director                                               cast  \
0                  NaN                                    